In [14]:
#Instala os pacotes básicos
import pandas as pd
try:
    import pandas_ta as ta
except:
    !pip install pandas_ta
    import pandas_ta as ta
import datetime as dt
import math
import matplotlib
import plotly.graph_objects as go
from plotly.subplots import make_subplots 
import numpy as np

from IPython.display import display

import stockstats as sts

import talib

#Constantes e parâmetros
DIRETORIO_DADOS = './data/'         #Diretório dos arquivos de dados (leitura)
DIRETORIO_SAIDA = './output/'       #Diretório dos arquivos de saída (escrita)
TICKET = 'EURUSD'                   #Moeda (par)
TIMEFRAME = 240                     #Tempo gráfico
INICIO = '1999-01-01'               #Data inicial
FINAL = '2024-04-01'                #Data final

ATR_PERIOD = 14                     #Período do ATR   
EMA_FAST = 50                       #Período da EMA rápida
EMA_MEAN = 144                      #Período da EMA média    
EMA_SLOW = 200                      #Período da EMA lenta
RSI_PERIOD = 14                     #Período do RSI 

TAM_STOP = 1                        #Tamanho do stop loss em ATRs
RISCO_RETORNO = 3                   #Relação risco-retorno 
TOTAL_TAXAS = 0.0001                #Soma da taxa, spread e slipagge

In [15]:
#Carrega a base de dados, define o nome das colunas, cria o índcie e remove os registros que não estão no período solicitado
#Reforna o dataframe com os dados carregados no formato exigido pelas rotinas do sistema
def carregarArquivo(DIRETORIO, TICKET, TIMEFRAME, INICIO, FINAL):
    df_full = pd.read_csv(DIRETORIO+TICKET+str(TIMEFRAME)+'.csv')
    df_full.dropna(axis=0, how='all', inplace=True) #Remove todos os registros com null em algum campo
    df_full.columns=(['Date', 'Time', 'Open', 'High', 'Low', 'Close', 'Volume']) #Seta os cabeçalhos dos campos
    df_full = df_full[(df_full['High'] > df_full['Low']) & (df_full['Volume'] > 1)] #Remove candles sem negociação
    df_full['Open time'] = pd.to_datetime(df_full['Date'] + ' ' + df_full['Time'] + ':00+0300', format='%Y.%m.%d %H:%M:%S%z')
    df_full.drop(df_full.iloc[:, 0:2], axis = 1, inplace=True) #Remove os campos date e time
    df_full.set_index('Open time', inplace = True) #Cria o índice
    df = df_full[(df_full.index >= pd.to_datetime(INICIO+' 00:00:00+0300')) & (df_full.index <= pd.to_datetime(FINAL+' 00:00:00+0300'))]
    del[df_full] #Deleta a base full
    return df #Retorna a base com os registros do período selecionado
# #end def

In [16]:
df = carregarArquivo(DIRETORIO_DADOS, TICKET, TIMEFRAME, INICIO, FINAL)

In [17]:
df['RSI'] = ta.rsi(close=df['Close'], length=RSI_PERIOD)


stock = sts.StockDataFrame.retype(df)
stock['rsi_14']

df['RSI_Talib'] = talib.RSI(df['close'], timeperiod=14)
# df.dropna(inplace = True)

In [19]:
df.to_csv('RSI.csv')